# Tickers Module

This notebook demonstrates the ticker utilities from `specparser.amt`.

**Note:** The AMT module is organized into focused submodules:
- **`tickers.py`** - High-level ticker filtering for straddles (calls `asset_straddle_tickers`)
- **`asset_straddle_tickers.py`** - Core ticker computation with smart caching
- **`chain.py`** - Futures ticker normalization (BBG chain lookup)
- **`prices.py`** - Price fetching, caching, and DuckDB access
- **`valuation.py`** - Actions, pricing models, and PnL calculations

## Table of Contents

1. [Core Ticker Functions](#1.-Core-Ticker-Functions) - `get_asset_straddle_tickers`, `make_fut_ticker`, `split_ticker`
2. [Understanding Ticker Types](#2.-Understanding-Ticker-Types) - Vol, Hedge tickers
3. [Futures Ticker Generation](#3.-Futures-Ticker-Generation) - `make_fut_ticker`, month maps
4. [Normalized vs Actual Tickers](#4.-Normalized-vs-Actual-Tickers) - `fut_norm2act`, `fut_act2norm`
5. [Split Tickers](#5.-Split-Tickers-(Ticker-Transitions)) - `split_ticker` for transitions
6. [Finding Tickers Across Periods](#6.-Finding-Tickers-Across-Periods) - `find_assets_straddles_tickers`
7. [Straddle Tickers](#7.-Straddle-Tickers) - `filter_tickers`
8. [Getting Prices for Straddles](#8.-Getting-Prices-for-Straddles) - `get_prices`, `actions`, `get_straddle_actions`
9. [Practical Examples](#9.-Practical-Examples) - Patterns, ticker inventory
10. [Summary](#10.-Summary) - Function reference tables
11. [Benchmark](#11.-Benchmark) - Performance benchmarks

---

## Architecture

The ticker system has a clear hierarchy:

```
asset_straddle_tickers.py  <- Source of truth for ticker computation
    |
    +-- get_asset_straddle_tickers()  <- Single asset/straddle
    +-- find_assets_straddles_tickers()  <- Multiple assets, date range
    +-- make_fut_ticker()  <- Futures ticker generation
    +-- split_ticker()  <- Handle ticker transitions
    |
tickers.py  <- Thin wrapper for backward compatibility
    |
    +-- filter_tickers()  <- Public API used by prices.py
```

## Ticker Output Format

Tickers are returned as tables with columns:
- **`get_asset_straddle_tickers`**: `['name', 'ticker', 'field']`
- **`filter_tickers`**: `['asset', 'straddle', 'param', 'source', 'ticker', 'field']`

## Ticker Sources

| Source | Description |
|--------|-------------|
| `BBG` | Bloomberg direct ticker |
| `CV` | CitiVelocity (for swaption vols) |
| `calc` | Computed tickers (for swap hedges) |

## Split Ticker Format

When tickers change over time (e.g., LIBOR -> SOFR transition):
```
ticker1:YYYY-MM:ticker2
```
- Before the date: use `ticker1`
- At/after the date: use `ticker2`

In [ ]:
# Setup: Imports and helper function
import pandas as pd
from specparser.amt import (
    # Core ticker functions (from asset_straddle_tickers.py)
    get_asset_straddle_tickers,
    find_assets_straddles_tickers,
    make_fut_ticker,
    split_ticker,
    asset_straddle_ticker_key,
    # Straddle tickers (from tickers.py)
    filter_tickers,
    clear_ticker_caches,
    # Chain functions (from chain.py)
    fut_norm2act,
    fut_act2norm,
    clear_chain_caches,
    # Price functions (from prices.py)
    get_prices,
    load_all_prices,
    # Valuation functions (from valuation.py)
    actions,
    get_straddle_actions,
    get_straddle_valuation,
    # Table utilities (from table.py)
    table_to_rows,
    table_column,
    print_table,
    show_table,
    # Loader functions (for examining asset config)
    get_asset,
    find_assets,
)

# Data paths (relative to this notebook)
AMT_PATH = "../data/amt.yml"
PRICES_PATH = "../data/prices.parquet"
CHAIN_PATH = None  # "../data/futs.csv"
OVERRIDE_PATH = "../data/overrides.csv"

---
## 1. Core Ticker Functions

### `get_asset_straddle_tickers(asset, strym, ntrc, amt_path)`

Get all tickers for a single asset's straddle. Returns a table with columns:
- `name`: Parameter name (`vol`, `hedge`, `hedge1`, etc.)
- `ticker`: The ticker symbol
- `field`: The data field to fetch

**Parameters:**
- `asset`: Asset underlying (e.g., "CL Comdty")
- `strym`: Straddle year-month as "YYYY-MM"
- `ntrc`: Entry code ("N" for near, "F" for far)
- `amt_path`: Path to AMT YAML file

In [ ]:
# Get tickers for a commodity asset straddle
tickers = get_asset_straddle_tickers("CL Comdty", "2024-06", "N", AMT_PATH)
print("CL Comdty tickers for 2024-06, Near entry:")
show_table(tickers)

In [ ]:
# Get tickers for an equity index
tickers = get_asset_straddle_tickers("SPX Index", "2024-06", "N", AMT_PATH)
print("SPX Index tickers for 2024-06:")
show_table(tickers)

In [ ]:
# Get tickers for a currency asset
tickers = get_asset_straddle_tickers("EURUSD Curncy", "2024-06", "N", AMT_PATH)
print("EURUSD Curncy tickers for 2024-06:")
show_table(tickers)

In [ ]:
# Compare Near vs Far entry codes
print("CL Comdty - Near entry (N):")
show_table(get_asset_straddle_tickers("CL Comdty", "2024-06", "N", AMT_PATH))

print("\nCL Comdty - Far entry (F):")
show_table(get_asset_straddle_tickers("CL Comdty", "2024-06", "F", AMT_PATH))

### `find_assets_straddles_tickers(pattern, ntry, xpry, amt_path)`

Find unique tickers for multiple assets over a date range. Returns deduplicated results.

In [ ]:
# Find tickers for all commodities in 2024
commodity_tickers = find_assets_straddles_tickers("Comdty$", "2024-01", "2024-12", AMT_PATH)
print(f"Found {len(commodity_tickers['rows'])} unique ticker rows for commodities in 2024")
show_table(commodity_tickers)

In [ ]:
# Find tickers for specific assets
tickers = find_assets_straddles_tickers("^(CL|GC|SI) ", "2024-01", "2024-06", AMT_PATH)
print(f"CL, GC, SI tickers for H1 2024: {len(tickers['rows'])} rows")
show_table(tickers)

---
## 2. Understanding Ticker Types

### Vol Tickers
- **Sources**: `BBG`, `CV` (CitiVelocity), `BBG_LMEVOL`
- **Name**: `vol`
- **Purpose**: Get implied volatility for option pricing

### Hedge Tickers
- **Sources**: `BBG` (nonfut), `fut` (futures), `calc` (computed), `cds`
- **Names**: `hedge`, `hedge1`, `hedge2`, `hedge3`, `hedge4`
- **Purpose**: Get prices for hedging instruments

In [ ]:
# Look at the asset configuration to understand ticker sources
asset_data = get_asset(AMT_PATH, "CL Comdty")

print("CL Comdty configuration:")
print()
print("Vol config:")
for k, v in asset_data.get("Vol", {}).items():
    print(f"  {k}: {v}")
print()
print("Hedge config:")
for k, v in asset_data.get("Hedge", {}).items():
    print(f"  {k}: {v}")

In [ ]:
# Analyze hedge sources across different asset types
assets_table = find_assets(AMT_PATH, ".", live_only=True)
assets = table_column(assets_table, "asset")

hedge_sources = {}
for asset in assets:
    data = get_asset(AMT_PATH, asset)
    if data and "Hedge" in data:
        source = data["Hedge"].get("Source", "unknown")
        hedge_sources.setdefault(source, []).append(asset)

print("Hedge sources:")
for source, asset_list in sorted(hedge_sources.items()):
    print(f"  {source}: {len(asset_list)} assets")
    for a in asset_list[:3]:
        print(f"    - {a}")
    if len(asset_list) > 3:
        print(f"    ... and {len(asset_list) - 3} more")

---
## 3. Futures Ticker Generation

### `make_fut_ticker(fut_code, fut_month_map, min_year_offset, market_code, qualifier, year, month)`

Generate a futures ticker for a specific year/month.

**Month map:** 12 characters mapping Jan-Dec to contract codes:
- Standard: `FGHJKMNQUVXZ` (F=Jan, G=Feb, H=Mar, J=Apr, K=May, M=Jun, N=Jul, Q=Aug, U=Sep, V=Oct, X=Nov, Z=Dec)

In [ ]:
# Generate crude oil futures tickers
print("CL Comdty futures for 2024:")
print("-" * 40)
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
for month_num, month_name in enumerate(months, 1):
    ticker = make_fut_ticker(
        fut_code="CL",
        fut_month_map="GHJKMNQUVXZF",  # CL uses non-standard month map
        min_year_offset=0,
        market_code="Comdty",
        qualifier="",
        year=2024,
        month=month_num,
    )
    print(f"{month_name:3s}: {ticker}")

In [ ]:
# Futures with qualifier (e.g., 'R' for LME vol options)
print("LME Aluminum vol futures (with 'R' qualifier):")
for month_num in [1, 3, 6, 9, 12]:
    ticker = make_fut_ticker(
        fut_code="LA",
        fut_month_map="FGHJKMNQUVXZ",
        min_year_offset=0,
        market_code="Comdty",
        qualifier="R",  # Vol option qualifier
        year=2024,
        month=month_num,
    )
    print(f"  Month {month_num:2d}: {ticker}")

### Understanding Month Maps

Different assets have different delivery months. The month map encodes this:

| Position | Standard | Meaning |
|----------|----------|--------|
| 1 (Jan)  | F        | January contract |
| 2 (Feb)  | G        | February contract |
| 3 (Mar)  | H        | March contract |
| ...      | ...      | ... |
| 12 (Dec) | Z        | December contract |

Some assets only trade certain months (e.g., quarterly). In these cases, the month map repeats characters to indicate which contract to use.

In [ ]:
# Example: An asset with quarterly contracts only (H, M, U, Z)
print("Quarterly contract example (HHHMMMUUUZZZ):")
for month_num, month_name in enumerate(months, 1):
    ticker = make_fut_ticker(
        fut_code="ED",
        fut_month_map="HHHMMMUUUZZZ",
        min_year_offset=0,
        market_code="Comdty",
        qualifier="",
        year=2024,
        month=month_num,
    )
    print(f"{month_name:3s}: {ticker}")

---
## 4. Normalized vs Actual Tickers

Bloomberg uses specific ticker formats that may differ from the "normalized" format we generate.

**Example:**
- Normalized: `CLN2024 Comdty`
- Actual BBG: `CL N24 Comdty`

The `fut_norm2act()` and `fut_act2norm()` functions convert between these formats using a CSV lookup table.

In [ ]:
# Check if chain CSV exists and demonstrate conversion
import os

if CHAIN_PATH and os.path.exists(CHAIN_PATH):
    print(f"Chain CSV found: {CHAIN_PATH}")
    
    # Example conversion
    normalized = "CLN2024 Comdty"
    actual = fut_norm2act(CHAIN_PATH, normalized)
    print(f"Normalized: {normalized}")
    print(f"Actual:     {actual}")
    
    # Reverse conversion
    if actual:
        back = fut_act2norm(CHAIN_PATH, actual)
        print(f"Back:       {back}")
else:
    print(f"Chain CSV not configured: {CHAIN_PATH}")
    print("Skipping normalized/actual conversion examples.")

In [ ]:
# Clear the lookup cache if needed
clear_chain_caches()
print("Chain caches cleared")

---
## 5. Split Tickers (Ticker Transitions)

When Bloomberg changes ticker symbols, we use "split tickers" to handle the transition:

```
ticker1:YYYY-MM:ticker2
```

The `split_ticker()` function selects the appropriate ticker based on the date:
- If `split_date < current_date`: use `ticker1` (pre-split)
- Otherwise: use `ticker2` (post-split)

In [ ]:
# Example: LIBOR to SOFR transition
split = "USSWAP5 CMPN Curncy:2023-06:USOSFR5 Curncy"

print(f"Split ticker: {split}")
print()

# Before the split date
result = split_ticker(split, 2023, 5)  # May 2023
print(f"At 2023-05 (before split): {result}")

# At the split date
result = split_ticker(split, 2023, 6)  # June 2023
print(f"At 2023-06 (at split):     {result}")

# After the split date
result = split_ticker(split, 2023, 7)  # July 2023
print(f"At 2023-07 (after split):  {result}")

In [ ]:
# A non-split ticker returns unchanged
regular = "SPY US Equity"
result = split_ticker(regular, 2024, 6)
print(f"Regular ticker: {regular}")
print(f"Result:         {result}")

---
## 6. Finding Tickers Across Periods

### `find_assets_straddles_tickers(pattern, ntry, xpry, amt_path)`

Get unique tickers for multiple assets across a date range. This is useful for:
- Building a data request for Bloomberg
- Understanding data requirements for a backtest period
- Auditing ticker coverage

In [ ]:
# Get all unique tickers for commodities in 2024
tickers = find_assets_straddles_tickers("Comdty$", "2024-01", "2024-12", AMT_PATH)
print(f"Unique commodity tickers for 2024: {len(tickers['rows'])} rows")
show_table(tickers)

In [ ]:
# Get tickers for all live assets over a multi-year period
all_tickers = find_assets_straddles_tickers(".", "2020-01", "2024-12", AMT_PATH)
print(f"All unique tickers (2020-2024): {len(all_tickers['rows'])} rows")

---
## 7. Straddle Tickers

### `filter_tickers(asset, year, month, i, amt_path, chain_path)`

Get tickers for a specific straddle component with full context.

**Returns columns:**
- `asset`: The underlying asset
- `straddle`: Full straddle string
- `param`: Parameter name (`vol`, `hedge`, etc.)
- `source`: Data source (`BBG`, `CV`, `calc`)
- `ticker`: The ticker symbol
- `field`: The data field

**Parameters:**
- `i`: Straddle component index (0, 1, 2, ... for multiple schedule components)

In [ ]:
# Get straddle tickers for CL Comdty, June 2024, first component
tickers = filter_tickers(
    asset="CL Comdty",
    year=2024,
    month=6,
    i=0,  # First straddle component
    amt_path=AMT_PATH,
    chain_path=CHAIN_PATH
)
print("Straddle tickers for CL Comdty, June 2024, component 0:")
show_table(tickers)

In [ ]:
# Compare different straddle components (i=0 vs i=1)
for i in range(2):
    tickers = filter_tickers(
        asset="CL Comdty",
        year=2024,
        month=6,
        i=i,
        amt_path=AMT_PATH,
        chain_path=CHAIN_PATH
    )
    print(f"Component {i}: {len(tickers['rows'])} ticker rows")
    # Show the straddle string
    if tickers['rows']:
        straddle = tickers['rows'][0][1]  # straddle is column 1
        print(f"  Straddle: {straddle}")
    print()

---
## 8. Getting Prices for Straddles

The AMT module provides a pipeline for getting daily prices with action/strike columns.

**From `prices.py`:**
1. **`get_prices()`** - Get daily prices for a straddle

**From `valuation.py`:**
2. **`actions()`** - Add action, model, and strike columns
3. **`get_straddle_actions()`** - Convenience function: `get_prices()` + `actions()`
4. **`get_straddle_valuation()`** - Add valuation columns (mv, delta, pnl)

In [ ]:
# Load prices into memory for fast lookups
load_all_prices(PRICES_PATH)
print("Prices loaded")

In [ ]:
# Get daily prices for a straddle
prices_table = get_prices(
    "CL Comdty",
    2024,
    6,
    0,  # First straddle component
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH
)
print(f"Prices table: {len(prices_table['rows'])} rows")
print(f"Columns: {prices_table['columns']}")
show_table(prices_table)

In [ ]:
# Convenience function: get_prices + actions in one call
full_table = get_straddle_actions(
    "CL Comdty",
    2024,
    6,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)
print(f"Full straddle actions: {len(full_table['rows'])} rows")
show_table(full_table)

In [ ]:
# Get straddle valuation with PnL columns
valuation_table = get_straddle_valuation(
    "CL Comdty",
    2024,
    6,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)
print(f"Valuation columns: {valuation_table['columns']}")
show_table(valuation_table)

### Example: Different Asset Types

In [ ]:
# Currency straddle
currency_actions = get_straddle_actions(
    "EURUSD Curncy",
    2024,
    1,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)
print(f"EURUSD Curncy: {len(currency_actions['rows'])} rows")
show_table(currency_actions)

In [ ]:
# Metal commodity (LA = Aluminum)
metal_actions = get_straddle_actions(
    "LA Comdty",
    2024,
    1,
    0,
    AMT_PATH,
    CHAIN_PATH,
    PRICES_PATH,
    OVERRIDE_PATH
)
print(f"LA Comdty: {len(metal_actions['rows'])} rows")
show_table(metal_actions)

---
## 9. Practical Examples

### Example 1: Analyze Hedge Sources by Asset Class

In [ ]:
# Count hedge sources by asset class pattern
from collections import Counter

patterns = [
    ("Comdty$", "Commodities"),
    ("Index$", "Indices"),
    ("Curncy$", "Currencies"),
    ("Equity$", "Equities"),
]

print("Hedge sources by asset class:")
print("-" * 50)

for pattern, name in patterns:
    assets_table = find_assets(AMT_PATH, pattern, live_only=True)
    assets = table_column(assets_table, "asset")
    
    sources = Counter()
    for asset in assets:
        data = get_asset(AMT_PATH, asset)
        if data and "Hedge" in data:
            sources[data["Hedge"].get("Source", "unknown")] += 1
    
    if sources:
        print(f"\n{name}:")
        for source, count in sources.most_common():
            print(f"  {source}: {count}")

### Example 2: Find Assets with Split Tickers

In [ ]:
# Find assets that have ticker transitions
import re

assets_table = find_assets(AMT_PATH, ".", live_only=True)
assets = table_column(assets_table, "asset")

split_assets = []
for asset in assets:
    data = get_asset(AMT_PATH, asset)
    if not data:
        continue
    
    # Check Hedge for split tickers
    hedge = data.get("Hedge", {})
    for key in ["Ticker", "hedge", "hedge1"]:
        ticker = hedge.get(key, "")
        if isinstance(ticker, str) and re.search(r':\d{4}-\d{2}:', ticker):
            split_assets.append((asset, key, ticker))

if split_assets:
    print(f"Assets with split tickers ({len(split_assets)}):")
    for asset, key, ticker in split_assets:
        print(f"  {asset}")
        print(f"    {key}: {ticker}")
else:
    print("No assets with split tickers found")

### Example 3: Build a Ticker Inventory

In [ ]:
# Build a summary of all unique tickers needed for 2024
all_tickers = find_assets_straddles_tickers(".", "2024-01", "2024-12", AMT_PATH)

# Group by field pattern
from collections import defaultdict

by_field = defaultdict(set)
for row in all_tickers["rows"]:
    name, ticker, field = row
    by_field[field].add(ticker)

print("Ticker inventory for 2024:")
print("-" * 40)
for field in sorted(by_field.keys()):
    tickers = by_field[field]
    if field == "":
        field_display = "(calc - no BBG field)"
    elif field == "none":
        field_display = "(CV - no BBG field)"
    else:
        field_display = field
    print(f"\n{field_display}: {len(tickers)} unique tickers")
    # Show first 3 examples
    for t in list(tickers)[:3]:
        print(f"  - {t}")
    if len(tickers) > 3:
        print(f"  ... and {len(tickers) - 3} more")

---
## 10. Summary

### Module Organization

| Module | Purpose |
|--------|--------|
| `asset_straddle_tickers.py` | Core ticker computation with smart caching |
| `tickers.py` | High-level wrapper (`filter_tickers`) |
| `chain.py` | Futures ticker normalization |
| `prices.py` | Price fetching and caching |
| `valuation.py` | Actions, pricing models, PnL |

### Ticker Functions

| Function | Description |
|----------|-------------|
| `get_asset_straddle_tickers(asset, strym, ntrc, path)` | Get tickers for single asset/straddle |
| `find_assets_straddles_tickers(pattern, ntry, xpry, path)` | Find unique tickers for multiple assets |
| `make_fut_ticker(...)` | Generate futures ticker |
| `split_ticker(ticker, year, month)` | Handle ticker transitions |
| `asset_straddle_ticker_key(...)` | Get cache key (for debugging) |
| `filter_tickers(...)` | Get tickers with full straddle context |
| `clear_ticker_caches()` | Clear all ticker caches |

### Chain Functions

| Function | Description |
|----------|-------------|
| `fut_norm2act(csv, ticker)` | Normalized -> actual ticker |
| `fut_act2norm(csv, ticker)` | Actual -> normalized ticker |
| `clear_chain_caches()` | Clear chain lookup cache |

### Output Formats

| Function | Columns |
|----------|--------|
| `get_asset_straddle_tickers` | `['name', 'ticker', 'field']` |
| `find_assets_straddles_tickers` | `['name', 'ticker', 'field']` |
| `filter_tickers` | `['asset', 'straddle', 'param', 'source', 'ticker', 'field']` |

---
## 11. Benchmark

Benchmark the ticker extraction functions.

In [ ]:
# Benchmark: Get all unique tickers for all live assets (2001-2025)
import timeit

# Parameters
START_YM, END_YM = "2001-01", "2025-12"
PATTERN = "."
N_RUNS = 3

# Warmup
clear_ticker_caches()
_ = find_assets_straddles_tickers(PATTERN, START_YM, END_YM, AMT_PATH)

# Get result stats
result = find_assets_straddles_tickers(PATTERN, START_YM, END_YM, AMT_PATH)
n_rows = len(result['rows'])

print(f"All live assets ({START_YM} to {END_YM}): {n_rows} unique ticker rows")
print()

# Benchmark (with cache - typical use case)
times_cached = timeit.repeat(
    lambda: find_assets_straddles_tickers(PATTERN, START_YM, END_YM, AMT_PATH),
    repeat=N_RUNS,
    number=1
)

# Benchmark (without cache - cold start)
def cold_run():
    clear_ticker_caches()
    return find_assets_straddles_tickers(PATTERN, START_YM, END_YM, AMT_PATH)

times_cold = timeit.repeat(cold_run, repeat=N_RUNS, number=1)

print(f"Benchmark ({N_RUNS} runs):")
print("-" * 50)
print(f"{'With cache (warm):':<25} {min(times_cached)*1000:6.1f}ms")
print(f"{'Without cache (cold):':<25} {min(times_cold)*1000:6.1f}ms")